In [1]:
import pandas as pd

#Import the AAR Data
file_path = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Test/AAR.xlsx'
df_cargo = pd.read_excel(file_path)

In [2]:
# Calculate the percentage for each row excluding the 'Commodity' column
df_percentage = df_cargo.iloc[:, 1:].div(df_cargo.iloc[:, 1:].sum(axis=1), axis=0) * 100
df_percentage.insert(0, 'Commodity', df_cargo['Commodity'])

In [3]:
#Import Census Data
file_path_block_census = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/pdb2023bg- Planning Group Track - Indiana Filtered.xlsx'
df_census = pd.read_excel(file_path_block_census)

In [4]:
#Import Occupational Data
file_path_occupation = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/ACSST5Y2022.S2405_2024-10-27T152508/ACSST5Y2022.S2405-Data_edited_3.xlsx'
df_occupation = pd.read_excel(file_path_occupation)

In [5]:
# Convert GIDTR in df_census to string
df_census['GIDTR'] = df_census['GIDTR'].astype(str)

# Extract the last 11 digits from GEO_ID 
df_occupation['GIDTR'] = df_occupation['Geography'].str[-11:].astype(str)

# Merge the two DataFrames on the GIDTR column
merged_df = pd.merge(df_census, df_occupation, on='GIDTR', how='inner')

In [6]:
# List of industry columns 
industry_columns = [
    'Agriculture, forestry, fishing and hunting, and mining',
    'Construction',
    'Manufacturing',
    'Wholesale trade',
    'Retail trade',
    'Transportation and warehousing, and utilities',
    'Information',
    'Finance and insurance, and real estate and rental and leasing',
    'Professional, scientific, and management, and administrative and waste management services',
    'Educational services, and health care and social assistance',
    'Arts, entertainment, and recreation, and accommodation and food services',
    'Other services, except public administration',
    'Public administration'
]

# Calculate the total number of workers for each industry across all tracts
industry_totals = merged_df[industry_columns].sum()

# Calculate the percentage of workers in each industry for each census tract
for column in industry_columns:
    percentage_column = f"{column}_Percentage"  # New column name for percentage
    merged_df[percentage_column] = merged_df[column] / industry_totals[column]

In [7]:
# Mapping of commodities to their relevant industries
industry_commodity_map = {
    'Agriculture, forestry, fishing and hunting, and mining': ['Farm Products', 'Food Products', 'Coal', 'Primary Metal Products'],
    'Construction': ['Primary Metal Products'],
    'Manufacturing': ['Chemicals', 'Petroleum', 'Coal', 'Primary Metal Products'],
    'Wholesale trade': ['Other'],
    'Retail trade': ['Other'],
    'Transportation and warehousing, and utilities': ['Coal', 'Primary Metal Products'],
    'Professional, scientific, and management, and administrative and waste management services': ['Waste and Scrap'],
    'Arts, entertainment, and recreation, and accommodation and food services': ['Food Products'],
}

# Initialize columns for each commodity's distribution
for commodity in df_cargo.columns[1:]: 
    merged_df[f'{commodity}_distributed'] = 0

# Loop through each commodity in df_cargo to distribute
for commodity in df_cargo.columns[1:]:
    relevant_industries = [industry for industry, com_list in industry_commodity_map.items() if commodity in com_list]
    relevant_industry_population = merged_df[relevant_industries].sum(axis=1).sum()
    merged_df[f'{commodity}_percentage'] = merged_df[relevant_industries].sum(axis=1) / relevant_industry_population
    merged_df[f'{commodity}_distributed'] = merged_df[f'{commodity}_percentage'] * df_cargo[commodity][0]

# Drop the temporary percentage columns
percentage_columns = [f'{commodity}_percentage' for commodity in df_cargo.columns[1:]]

In [8]:
#Import List of Latitude and Longitude Coordinates for each census track
file_path_coor = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Test/Coordinates.xlsx'
df_coor = pd.read_excel(file_path_coor) 

In [9]:
# Convert both 'GIDTR' and 'GEOID' to strings
merged_df['GIDTR'] = merged_df['GIDTR'].astype(str)
df_coor['GEOID'] = df_coor['GEOID'].astype(str)

# Merge the coordinate data into the data set
merged_df = pd.merge(merged_df, df_coor[['GEOID', 'Latitude', 'Longitude']], 
                              left_on='GIDTR', right_on='GEOID', how='left')

In [10]:
#Import SVI Data
file_path_svi = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/Indiana_censusTract.csv'
df_svi = pd.read_csv(file_path_svi)

In [11]:
# Ensure both columns are the same type (e.g., string)
merged_df['GEOID'] = merged_df['GEOID'].astype(str)
df_svi['FIPS'] = df_svi['FIPS'].astype(str)

# Merging SVI data into the data set
merged_df = merged_df.merge(df_svi[['FIPS', 'RPL_THEMES']], 
                             left_on='GIDTR', 
                             right_on='FIPS', 
                             how='left')

In [12]:
import geopandas as gpd
import numpy as np
import zipfile

In [13]:
import folium
from folium.plugins import HeatMap

In [14]:
from folium.plugins import MarkerCluster

#Primary Metal Distribution
# Create the base map
m = folium.Map(location=[40.0, -85.0], zoom_start=7)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Primary Metal Products_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("primary_metal_products_clustered_map.html")


In [15]:
#Coal Distribution
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Coal_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Coal_distributed_products_clustered_map.html")


In [16]:
#Farm Products
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Farm Products_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Farm Products_distributed_map.html")

In [17]:
#Food Products
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Food Products_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Food Products_distributed.html")

In [18]:
#Chemical Products
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Chemicals_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Chemicals_distributed.html")

In [19]:
#Other
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Other_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Other_distributed.html")

In [20]:
#Waste and Scrap
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Waste and Scrap_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Waste and Scrap_distributed.html")

In [21]:
#Petroleum
# Loop through the data and add markers to the cluster
for _, row in merged_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    value = row['Petroleum_distributed']
    
    # Create a popup with value information
    popup = folium.Popup(f"Value: {value:.2f}", max_width=300)
    
    # Add the marker with a circle marker icon
    folium.CircleMarker(
        location=[lat, lon],
        radius=value / 10,  # Adjust the divisor for desired size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("Petroleum_distributed.html")

The below data analysis uses the files found in the City Estimates file and are used to estimate terminated and originated commodities shipped throughout the US. This includes cargo that is terminated, originated and pass through the state of Indiana. 

In [22]:
with zipfile.ZipFile("/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/tl_2024_us_county.zip", "r") as zip_ref:
    zip_ref.extractall("tl_2024_us_county")

In [23]:
counties = gpd.read_file("tl_2024_us_county/tl_2024_us_county.shp")
counties.to_file('counties.geojson', driver = 'GeoJSON')

In [24]:
cmd_df = pd.read_csv('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/Indiana_Rail_FAF.csv')

In [25]:
#Aggregate all data with a destination in Indiana
#USING 2022 as 2023 estimates are not yet finalized
cmd_dms_orig_df = cmd_df.groupby(['dms_orig', 'sctg2']).\
                agg(
                    {'tons_2022': 'sum'})\
                .reset_index()\
                .rename(columns = {'dms_orig' : 'dms_loc'})

cmd_dms_orig_df = cmd_dms_orig_df[(cmd_dms_orig_df['dms_loc']>= 180) & (cmd_dms_orig_df['dms_loc'] <= 189)].copy(deep = True)

#Aggregate all dadta with an origin from Indiana
cmd_dms_dest_df = cmd_df.groupby(['dms_dest', 'sctg2'])\
                .agg(
                    {'tons_2022': 'sum'})\
                .reset_index()\
                .rename(columns = {'dms_dest' : 'dms_loc'})

cmd_dms_dest_df = cmd_dms_dest_df[(cmd_dms_dest_df['dms_loc']>= 180) & (cmd_dms_dest_df['dms_loc'] <= 189)].copy(deep = True)




In [26]:
#Combine datasets to create city level commodity shares
cmd_dms_loc_df = pd.concat([cmd_dms_orig_df, cmd_dms_dest_df], ignore_index=True)
cmd_dms_ref_df = cmd_dms_loc_df.groupby('dms_loc').agg({'tons_2022': 'sum'})\
                .reset_index()\
                .rename(columns = {'tons_2022': 'total_tons_2022'})


cmd_dms_loc_df = pd.merge(cmd_dms_loc_df, cmd_dms_ref_df, on = 'dms_loc', how = 'left')

cmd_dms_loc_df['loc_cmd_share'] = cmd_dms_loc_df['tons_2022']/cmd_dms_loc_df['total_tons_2022']

#City level commodity shares to be distributed to the city and collar counties?
#Preliminary logic below, but needs to be combined with employment data methodology and structure

In [27]:
IN_Counties = counties[counties['STATEFP'] == '18'].copy(deep = True)

#Primary and Collar Counties for listed cities
FW_Cnt = ['Allen', 'Wells', 'Whitley', 'Huntington', 'Adams']
Gary_Cnt = ['Lake', 'Porter']
Ind_Cnt = ['Marion', 'Boone', 'Hamilton', 'Madison', 'Hancock', 'Shelby', 'Johnson', 'Morgan', 'Hendricks']

FW_df = IN_Counties[IN_Counties['NAME'].apply(lambda x: any(s in x for s in FW_Cnt))]
Gary_df = IN_Counties[IN_Counties['NAME'].apply(lambda x: any(s in x for s in Gary_Cnt))]
Ind_df = IN_Counties[IN_Counties['NAME'].apply(lambda x: any(s in x for s in Ind_Cnt))]

IN_Counties['Commodity_Mapping'] = np.where(IN_Counties['NAME'].str.contains('|'.join(FW_Cnt), case = False), 'Fort Wayne IN',
                                            np.where(IN_Counties['NAME'].str.contains('|'.join(Gary_Cnt), case = False), 'Chicago IL-IN-WI (IN Part)',
                                                     np.where(IN_Counties['NAME'].str.contains('|'.join(Ind_Cnt), case = False), 'Indianapolis IN',
                                                              'Rest of IN')))

IN_Counties['dms_code'] = np.where(IN_Counties['NAME'].str.contains('|'.join(FW_Cnt), case = False), 183,
                                            np.where(IN_Counties['NAME'].str.contains('|'.join(Gary_Cnt), case = False), 181,
                                                     np.where(IN_Counties['NAME'].str.contains('|'.join(Ind_Cnt), case = False), 182,
                                                              189)))

In [28]:
data_df = pd.read_csv('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/FAF5.6.1_2018-2023/FAF5.6.1_2018-2023.csv')

In [29]:
#Load in refference dfs, filter down to just rail
city_df = pd.read_excel('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/FAF5.6.1_2018-2023/FAF5_metadata.xlsx', sheet_name = 'FAF Zone (Domestic)', header= 0)
comm_type_df = pd.read_excel('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/FAF5.6.1_2018-2023/FAF5_metadata.xlsx', sheet_name = 'Commodity (SCTG2)', header= 0)
dist_df = pd.read_excel('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/FAF5.6.1_2018-2023/FAF5_metadata.xlsx', sheet_name = 'Distance Band', header= 0)
rail_df = data_df[data_df['dms_mode'] == 2].copy(deep = True)
indiana_codes = [181, 182, 183, 189]

In [30]:
orig_df = city_df[['Numeric Label', 'Short Description']].rename(columns= {'Numeric Label': 'dms_orig', 'Short Description': 'dms_orig_desc'})
dest_df = city_df[['Numeric Label', 'Short Description']].rename(columns= {'Numeric Label': 'dms_dest', 'Short Description': 'dms_dest_desc'})
comm_type_df = comm_type_df.rename(columns= {'Numeric Label': 'sctg2', 'Description' : 'Commodity_Type'})
dist_df = dist_df.rename(columns = {'Numeric Label': 'dist_band', 'Description' : 'Distance_Band'})


In [31]:
rail_df = rail_df[rail_df['dms_orig'].isin(indiana_codes) | rail_df['dms_dest'].isin(indiana_codes)]
final_df = pd.merge(rail_df, orig_df, on = 'dms_orig', how = 'left')
final_df = pd.merge(final_df, dest_df, on = 'dms_dest', how = 'left')
final_df = pd.merge(final_df, comm_type_df, on = 'sctg2', how = 'left')
final_df = pd.merge(final_df, dist_df, on = 'dist_band', how = 'left')
final_df = final_df.drop(columns = ['fr_orig', 'fr_dest', 'fr_inmode', 'fr_outmode'])
final_df.to_csv('Indiana_Rail_FAF.csv')



In [32]:
# merge IN_Counties and cmd_dms_loc_df
cmd_dms_loc_df_2 = pd.merge(
    cmd_dms_loc_df,                # Left DataFrame
    IN_Counties[['dms_code', 'Commodity_Mapping','NAMELSAD']],  # Columns to merge from IN_Counties
    left_on='dms_loc',             # Key column in cmd_dms_loc_df
    right_on='dms_code',           # Key column in IN_Counties
    how='left'                     # Use left join to retain all rows in cmd_dms_loc_df
)


In [33]:
#Import Commodity Code Data set
file_path_code = '/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/Commodity_Codes.xlsx'
df_FAF_codes = pd.read_excel(file_path_code)  # Specify the sheet name if needed



In [34]:
# merge cmd_dms_loc_df and df_FAF_codes
cmd_dms_loc_df_3 = pd.merge(
    cmd_dms_loc_df_2,                # Left DataFrame
    df_FAF_codes[['Code', 'Commodity','AAR Category']],  
    left_on='sctg2',             
    right_on='Code',        
    how='left' 
)

In [35]:
# Pivot the data to make AAR Category commodity types into columns
commodity_pivot = cmd_dms_loc_df_3.pivot_table(
    index='NAMELSAD',                 # Rows will be counties
    columns='AAR Category',          # Columns will be commodity types
    values='tons_2022',              # Values to aggregate
    aggfunc='sum',                   # Summing weights of commodities
    fill_value=0                     # Replace NaN with 0
).reset_index()

# Rename columns for clarity (optional, flatten MultiIndex columns)
commodity_pivot.columns.name = None  # Remove the pivot table column grouping
commodity_pivot = commodity_pivot.rename_axis(None, axis=1)  # Remove row name

# Merge back into the original dataset
cmd_dms_loc_df_4 = pd.merge(
    cmd_dms_loc_df_3,
    commodity_pivot,
    on='NAMELSAD',
    how='left'
)


In [36]:
# Keep only the first instance of each county based on NAMELSAD to reduce redundancy
cmd_dms_loc_df_5 = cmd_dms_loc_df_4.drop_duplicates(subset='NAMELSAD', keep='first')



In [37]:
# merge into main data set cmd_dms_loc_df_5
merged_df_2 = pd.merge(
    merged_df,                # Left DataFrame
    cmd_dms_loc_df_5[['NAMELSAD', 'Chemicals', 'Coal', 'Farm Products', 'Food Products', 
    'Other', 'Petroleum', 'Primary Metal Products', 'Waste and Scrap','total_tons_2022']],  
    left_on='County_name',            
    right_on='NAMELSAD',          
    how='left'                 
)


In [38]:
#Import Pass through data
pass_through_df = pd.read_csv('/Users/allenho/Documents/Grad IU/Fall 2024/Data Science In Practice/Freight/Data Sets/Team/ind_passthrough_estimates.csv')

In [39]:
# merge pass through data df_FAF_codes
pass_through_df_2 = pd.merge(
    pass_through_df,                # Left DataFrame
    df_FAF_codes[['Code', 'Commodity','AAR Category']],  
    left_on='sctg2',             
    right_on='Code',        
    how='left' 
)

In [40]:
#Cleaning pass through data to find on a per county basis the amount in each category
pass_through_expanded = pass_through_df_2.copy()


pass_through_expanded['counties_crossed'] = pass_through_expanded['counties_crossed'].str.split(',')
pass_through_expanded = pass_through_expanded.explode('counties_crossed')

# Remove unwanted characters such as brackets and quotes, and strip any leading/trailing whitespace
pass_through_expanded['counties_crossed'] = pass_through_expanded['counties_crossed']\
    .str.replace(r"[\[\]']", "", regex=True)\
    .str.strip()

#Add word County for df merge later
pass_through_expanded['counties_crossed'] = pass_through_expanded['counties_crossed'].apply(
    lambda x: x if "County" in x else f"{x} County"
)


aggregated_df = pass_through_expanded.groupby(['counties_crossed', 'AAR Category'], as_index=False)['tons_2022'].sum()
result_df = aggregated_df.pivot(index='counties_crossed', columns='AAR Category', values='tons_2022')
result_df = result_df.fillna(0)
result_df.reset_index(inplace=True)


print(result_df)

AAR Category    counties_crossed    Chemicals         Coal  Farm Products  \
0                   Adams County  1480.530256  1069.396004    3902.589636   
1                   Allen County  1984.946953  1195.269354    4674.351890   
2             Bartholomew County   907.043141  1427.010352    6617.535316   
3                  Benton County  1358.743113  4436.743365     147.353922   
4               Blackford County   861.405525   972.980436     845.196035   
..                           ...          ...          ...            ...   
87             Washington County   692.967569   368.724177    1948.462812   
88                  Wayne County  1038.474108   290.780509    3216.943544   
89                  Wells County  1668.807401  3469.115316    3729.636148   
90                  White County  2023.268293  1771.994637     610.003549   
91                Whitley County   703.590281   329.469680    1001.133691   

AAR Category  Food Products        Other    Petroleum  Primary Metal Produc

In [41]:
# merge into main datasetthe above
merged_df_3 = pd.merge(
    merged_df_2,                # Left DataFrame
    result_df[['counties_crossed', 'Chemicals', 'Coal', 'Farm Products', 'Food Products', 
    'Other', 'Petroleum', 'Primary Metal Products', 'Waste and Scrap']],  
    left_on='County_name',            
    right_on='counties_crossed',          
    how='left'                     # Use left join to retain all rows in cmd_dms_loc_df
)


In [42]:
# List of columns to sum from pass through data
columns_to_sum = [
    'Chemicals_y', 'Coal_y', 'Farm Products_y', 'Food Products_y', 
    'Other_y', 'Petroleum_y', 'Primary Metal Products_y', 'Waste and Scrap_y'
]

#List of columns to sum from terminated and originated data
columns_to_sum_term = [
    'Chemicals_x', 'Coal_x', 'Farm Products_x', 'Food Products_x', 
    'Other_x', 'Petroleum_x', 'Primary Metal Products_x', 'Waste and Scrap_x'
]

merged_df_3['total_tons_2022_y'] = merged_df_3[columns_to_sum].sum(axis=1)

# Calculate percentages for each column and add new columns for pass through data
for col in columns_to_sum:
    merged_df_3[f'{col}_pct'] = (merged_df_3[col] / merged_df_3['total_tons_2022_y']) * 100
    

# Calculate percentages for each column and add new columns for terminated data
for col in columns_to_sum_term:
    merged_df_3[f'{col}_pct'] = (merged_df_3[col] / merged_df_3['total_tons_2022']) * 100


The below calculates a risk score for commodity data. As it currently stands the weight is distributed equally. Commodity categories with _distributed is derived from census data on worker distribution and original AAR categories. Categories with _x are derived from terminated/oringated data, and those denoted with _y are derived from pass through data. You can adjust the weight distribution as needed. 

In [43]:
# Define risk weights for each commodity
risk_weights = {
    'Chemicals': 5,
    'Coal': 3,
    'Farm Products': 1,
    'Food Products': 1,
    'Other': 2,
    'Petroleum': 4,
    'Primary Metal Products': 2,
    'Waste and Scrap': 4
}

# List of commodities
commodities = [
    'Chemicals', 'Coal', 'Farm Products', 'Food Products', 
    'Other', 'Petroleum', 'Primary Metal Products', 'Waste and Scrap'
]

# Weight for each dataset
dataset_weight = 1/3

# Initialize the total risk score column
merged_df_3['total_combined_risk'] = 0

# Calculate the combined risk score for each county
for commodity in commodities:
    # Get column names for each dataset
    distributed_col = f"{commodity}_distributed"
    terminated_col = f"{commodity}_x"
    pass_through_col = f"{commodity}_y"
    
    # Calculate normalized risk for each dataset
    distributed_risk = (merged_df_3[distributed_col] / merged_df_3[distributed_col].sum()) * dataset_weight
    terminated_risk = (merged_df_3[terminated_col] / merged_df_3[terminated_col].sum()) * dataset_weight
    pass_through_risk = (merged_df_3[pass_through_col] / merged_df_3[pass_through_col].sum()) * dataset_weight
    
    # Combine risks for the commodity and weight by risk factor
    combined_risk = (distributed_risk + terminated_risk + pass_through_risk) * risk_weights[commodity]
    
    # Add to total combined risk score
    merged_df_3['total_combined_risk'] += combined_risk

# Scale the total combined risk score between 1 and 10
min_risk = merged_df_3['total_combined_risk'].min()
max_risk = merged_df_3['total_combined_risk'].max()

merged_df_3['total_combined_risk'] = 1 + (merged_df_3['total_combined_risk'] - min_risk) / (max_risk - min_risk) * 9

# Display the resulting dataset
merged_df_3[['total_combined_risk']]


,total_combined_risk
0,5.653406
1,4.365178
2,5.533756
3,5.364302
4,5.371572
...,...
1691,3.304667
1692,4.672357
1693,4.939028
1694,3.027639


In [44]:
merged_df_3.to_excel("AAR_Analysis_City_Estimates.xlsx", index=False)

/var/folders/jh/7v3ft64505b60fvqz7vsgpz40000gn/T/ipykernel_3365/3992824761.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merged_df_3.to_excel("AAR_Analysis_City_Estimates.xlsx", index=False)
